In [3]:
import json
import requests
import pandas as pd
import sqlite3

In [4]:
def APIcall():    
    url = "https://rawg-video-games-database.p.rapidapi.com/games"

    headers = {
        'x-rapidapi-host': "rawg-video-games-database.p.rapidapi.com",
        'x-rapidapi-key': "ffff914b1amshea9ca31ea112264p1890dfjsnb6ed091db8e4"
        }

    response = requests.request("GET", url, headers=headers)

    data = response.text
    json_data = json.loads(data)
    return json_data

In [5]:
def sort_json(data, rows):
    results = data['results']
    for i in range(len(results)):
        row = []
        idx = results[i]
        ID = idx['id']
        name = idx['name']
        release_date = idx['released']
        rating = idx['rating']
        rating_count = idx['ratings_count']
        metacritic = idx['metacritic']
        avg_play = idx['playtime']
        genres = []
        for g in idx['genres']:
            genres.append(g['name'])
        platforms = []
        for p in idx['parent_platforms']:
            platforms.append(p['platform']['name'])
        online_stores = []
        for s in idx['stores']:
            online_stores.append(s['store']['name'])
        row.extend([ID, name, release_date, rating, rating_count, metacritic, avg_play, genres, platforms, online_stores])
        rows.append(row)
    return rows

In [6]:
json_data = APIcall()

In [7]:
dataset = []
def Nextcall(call, dataset):
    sort_json(call, dataset)
    for i in range(24):
        url = call['next']
        response = requests.request("GET", url)
        data = response.text
        json_data = json.loads(data)
        sort_json(json_data, dataset)
        call = json_data
    return dataset
final_data = Nextcall(json_data, dataset)

In [8]:
columns = ['ID', 'Title', 'Release_Date', 'Score', 'Score_Counts', 'Metascore', 'Average_Playtime_Hours',
          'Genre', 'Platforms', 'Online_Store_Sites']
df = pd.DataFrame(final_data, columns=columns)

In [9]:
# removal of list and string formatting from Genre column
df['Genre'] = df['Genre'].astype('str')
df['Genre'] = df['Genre'].str.replace(']', '')
df['Genre'] = df['Genre'].str.replace('[', '')
df['Genre'] = df['Genre'].str.replace("'", '')

# removal of list and string formatting from Platforms column
df['Platforms'] = df['Platforms'].astype('str')
df['Platforms'] = df['Platforms'].str.replace(']', '')
df['Platforms'] = df['Platforms'].str.replace('[', '')
df['Platforms'] = df['Platforms'].str.replace("'", '')

# removal of list and string formatting from Online Store Sites column
df['Online_Store_Sites'] = df['Online_Store_Sites'].astype('str')
df['Online_Store_Sites'] = df['Online_Store_Sites'].str.replace(']', '')
df['Online_Store_Sites'] = df['Online_Store_Sites'].str.replace('[', '')
df['Online_Store_Sites'] = df['Online_Store_Sites'].str.replace("'", '')

# cast release date as a datetime object
df['Release_Date'] = pd.to_datetime(df['Release_Date'])

# create variable for grouping once dataset is merged
df['Type'] = 'Game'

In [10]:
for col in df.columns:
    print(col, df[col].dtype)

ID int64
Title object
Release_Date datetime64[ns]
Score float64
Score_Counts int64
Metascore float64
Average_Playtime_Hours int64
Genre object
Platforms object
Online_Store_Sites object
Type object


In [35]:
with sqlite3.connect("Milestone5.db") as conn:
    cursor = conn.cursor()
    cursor.execute('DROP TABLE games')
    cursor.execute('CREATE TABLE games (ID integer, Title text, Release_Date integer, Score integer, Score_Counts integer, Metascore real, Average_Playtime_Hours integer, Genre text, Platforms text, Online_Store_Sites text, Type text)')
    # fill 'games' table with the values from pandas dataframe
    df.to_sql('games', conn, if_exists = 'replace', index = False)


In [1]:
# with sqlite3.connect("Milestone5.db") as conn:
#     cursor = conn.cursor()
#     cursor.execute('SELECT Title FROM books')
#     for row in cursor.fetchall():
#         print(row)

In [41]:
df.to_csv('games.csv', index = False)

In [11]:
df

,ID,Title,Release_Date,Score,Score_Counts,Metascore,Average_Playtime_Hours,Genre,Platforms,Online_Store_Sites,Type
0,3498,Grand Theft Auto V,2013-09-17,4.48,3900,97.0,68,"Action, Adventure","PC, PlayStation, Xbox","Epic Games, PlayStation Store, Xbox Store, Xbo...",Game
1,4200,Portal 2,2011-04-18,4.61,3309,95.0,11,"Shooter, Puzzle","PC, PlayStation, Xbox, Apple Macintosh, Linux","Xbox Store, Xbox 360 Store, PlayStation Store,...",Game
2,3328,The Witcher 3: Wild Hunt,2015-05-18,4.67,3578,93.0,51,"Action, Adventure, RPG","PC, PlayStation, Xbox, Nintendo","GOG, Xbox Store, Steam, PlayStation Store",Game
3,5286,Tomb Raider (2013),2013-03-05,4.06,2315,86.0,11,"Action, Adventure","PC, PlayStation, Xbox, Apple Macintosh","App Store, Google Play, PlayStation Store, Ste...",Game
4,5679,The Elder Scrolls V: Skyrim,2011-11-11,4.42,2777,94.0,44,"Action, RPG","PC, PlayStation, Xbox, Nintendo","Xbox 360 Store, Nintendo Store, Steam, PlaySta...",Game
...,...,...,...,...,...,...,...,...,...,...,...
495,19633,Need For Speed: Hot Pursuit,2010-11-16,3.83,427,86.0,3,"Action, Sports, Racing","PC, PlayStation, Xbox, iOS, Android, Nintendo","PlayStation Store, App Store, Google Play, Xbo...",Game
496,44525,Yakuza Kiwami,2016-01-21,4.03,440,83.0,6,"Action, Adventure, RPG, Fighting","PC, PlayStation, Xbox","Xbox Store, Steam, PlayStation Store",Game
497,2889,Oddworld: New 'n' Tasty,2014-07-22,3.55,272,87.0,4,"Action, Adventure, Indie, Platformer","PC, PlayStation, Xbox, iOS, Android, Apple Mac...","Google Play, App Store, Nintendo Store, GOG, S...",Game
498,18185,Company of Heroes: Tales of Valor,2009-04-07,3.32,140,70.0,1,Strategy,PC,Steam,Game
